In [ ]:
import pickle

import pandas as pd
from getpass import getpass

from melitk.fda2 import inventory

from app.data.utils.sparksql import SparkSQL
from app.data.utils.load_query import load_format
from app.conf.settings import DEFAULT_PARAMS

# Access to the databases

In [ ]:
melidata_user = "niacevedo"
melidata_pass = getpass()
spark = SparkSQL(user=melidata_user, password=melidata_pass)

# Data

## Read

In [ ]:
filename = '../src/app/data/creatives_performance/performance.sql'
query = load_format(filename, DEFAULT_PARAMS)

In [ ]:
df = spark.run_query(query)
df.head()

In [ ]:
df.shape

In [ ]:
df.sort_values(['cday', 'campaign_id', 'line_item_id', 'creative_id'], inplace = True)

## Processing

In [ ]:
df = df[~pd.isnull(df['campaign_id'])]
df = df[~pd.isnull(df['line_item_id'])]
df = df[~pd.isnull(df['creative_id'])]

## Save

In [ ]:
my_fury_new_token = 'dbe8df67db389e3155d6be40068da69e72413bb746d23bffe74762eb0fb2953b'
inventory.init(token = my_fury_new_token)

In [ ]:
# serialize
data_bytes = pickle.dumps(df)

# Create fda artifact
artifact_name = 'clicks_prints_per_day'
version = '1.0.0'
artifact = inventory.create_artifact(artifact_name, version = version, type_ = 'fda.Bytes')
artifact.save_from_bytes(data = data_bytes)